In [ ]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm

In [ ]:
BATCH_SIZE = 64
EPOCHS = 10
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
#Аугментация
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
LABELS = ['Tshirts', 'Shirts', 'Casual Shoes', 'Watches', 'Sports Shoes', 'Kurtas', 'Tops', 'Handbags', 'Heels', 'Sunglasses', 'Wallets', 'Flip Flops', 'Sandals', 'Briefs', 'Belts', 'Backpacks', 'Socks', 'Formal Shoes', 'Perfume and Body Mist', 'Jeans']
LABEL_TO_IDX = {label: idx for idx, label in enumerate(LABELS)}
IDX_TO_LABEL = {idx: label for label, idx in LABEL_TO_IDX.items()}

In [ ]:
class FashionDataset(Dataset):
    def __init__(self, df_meta, img_path, transform=None, is_test=False):
        self.df_meta = df_meta
        self.img_path = img_path
        self.transform = transform
        self.is_test = is_test

    def __len__(self):
        return len(self.df_meta)

    def __getitem__(self, idx):
        img_name = self.df_meta.iloc[idx, 0]
        img = Image.open(os.path.join(self.img_path, img_name)).convert('RGB')

        if self.transform:
            img = self.transform(img)

        img = img.to(DEVICE)

        if self.is_test:
            return img
        else:
            label = LABEL_TO_IDX[self.df_meta.iloc[idx, 1]]
            label = torch.tensor(label, dtype=torch.long).to(DEVICE)
            return img, label

In [ ]:
df_train = pd.read_csv('/kaggle/input/dl-5-image-classification/train-labels.csv')
df_train = df_train[df_train["image"] != "39401.jpg"]
df_test = pd.DataFrame({"image": os.listdir('/kaggle/input/dl-5-image-classification/test/test/')})

test_dataset = FashionDataset(df_test, '/kaggle/input/dl-5-image-classification/test/test', test_transforms, is_test=True)
train_dataset = FashionDataset(df_train, '/kaggle/input/dl-5-image-classification/train', train_transforms, is_test=False)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
model = models.resnet18(weights=None)
model.fc = nn.Linear(model.fc.in_features, 20)
model = model.to(DEVICE)

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

In [ ]:
def train():
    model.train()
    for epoch in range(EPOCHS):
        total_loss = 0
        correct = 0
        for images, labels in tqdm(train_loader, desc=f'Epoch {epoch+1}/{EPOCHS}'):
            optimizer.zero_grad()
            outputs = model(images)
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            correct += (outputs.argmax(1) == labels).sum().item()
        scheduler.step()
        print(f'Epoch {epoch+1}, Loss: {total_loss / len(train_loader)}, Accuracy: {correct / len(train_dataset):.4f}')

In [ ]:
def predict():
    model.eval()
    predictions = []
    with torch.no_grad():
        for images in tqdm(test_loader, desc='Predicting'):
            outputs = model(images)
            preds = outputs.argmax(1).cpu().numpy()
            predictions.extend(preds)

    df_test['label'] = [IDX_TO_LABEL[idx] for idx in predictions]
    df_test.to_csv('submission.csv', index=False)
    print('Predictions saved to submission.csv')

In [ ]:
train()
predict()

Для классификации изображений используется ResNet-18 – сверточная нейронная сеть (CNN), которая показала отличные результаты на задачах классификации.

Изменения в модели:

Взята стандартная ResNet-18 без предобученных весов (weights=None).
Последний полносвязный слой (fc) заменен на новый слой, соответствующий 20 классам.

Для улучшения обучения модели используются преобразования изображений (аугментация):

Для обучающего набора:
*   Изменение размера
*   Горизонтальное отражение
*   Изменение цветовых характеристик (яркость, контраст и т. д.)
*   Преобразование в тензор и нормализация

Для тестового набора:
*   Изменение размера
*   Нормализация

Модель обучается с использованием функции потерь CrossEntropyLoss и оптимизатора AdamW с регуляризацией.

После запуска на 1 эпохе Accuracy: 0.4470. Итоговый результат на kaggle 0.60236.

Далее увеличели количество эпох до 10 Accuracy: 0.7985. Итоговый результат на kaggle 0.83025.
